In [ ]:
import numpy as np
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from rich import print

from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import get_scorer
from xgboost import XGBRFRegressor

# from sklearn.tree import DecisionTreeRegressor

In [ ]:
df = pl.read_csv('../data/raw/chicago_properties.csv', null_values=['N/A', 'null'])
df.head()

In [ ]:
# Clean the price column
df = df.with_columns(
    pl.col('price')
    .replace({'$279,000+': '279000', 'Est. $138.8K': '138800', 'Est. $290K': '290000'})
    .str.replace('$', '', literal=True)
    .str.replace_all(',', '').cast(pl.Float32)
)

# Location of the property also matters, let's extract the zip code from the address
df = df.with_columns(
    pl.col('address')
    .str.extract(r'IL (\d{5})$')
    .alias('zipcode')
)

# Fill in the missing zip code (found using Google Search)
df = df.with_columns(
    zipcode=pl.when(pl.col('address')=="Madison FP Plan, Madison").then(pl.lit('60601')).otherwise(pl.col('zipcode'))
)

# Clean the square footage column, convert unknown values to null
df = df.with_columns(
    pl.col('square_footage').cast(pl.Float32, strict=False)
)

# Remove outliers
df = df.filter(
    ~pl.col('address').is_in([
        '1355 N Astor St, Chicago, IL 60610',
        '415 E North Water St #3205, Chicago, IL 60611'
    ]) # extremely high sq ft given less number of bedrooms
).filter(
    pl.col('zipcode') != '60602' # missing bedrooms for all properties in this zip code (2 rows)
)

df.head()

In [ ]:
df.describe()

In [ ]:
def build_model(model_params={}):
    """Build a model with the specified configuration

    Args:
        model_config (dict[str, Any]): Model configuration.

    Returns:
        object: Model object.
    """
    return XGBRFRegressor(**model_params, random_state=42)
    # return DecisionTreeRegressor(
    #     max_depth=16,
    #     min_samples_leaf=5,
    #     random_state=42,
    # )


def train(
    X,
    y,
    model_params={},
    cv=5,  # can also use train-test
    metrics=[],
    random_state=42,
):
    """Train a model and compute evaluation metrics.

    This function is supposed to do four things in order:
    1. Perform training along with the validation setup.
    2. Evaluate the model on specified metrics.
    3. Retrain the model on the full dataset.
    4. Save the model if a path is provided.

    Args:
        X (DataFrame): Features.
        y (DataFrame): Labels.
        model_params: Model parameters.
        cv (int, optional): Number of cross-validation folds. Defaults to 5.
        eval_metrics (list, optional): Evaluation metrics to compute. Defaults to [].

    Returns:
        object: Trained model.
        np.ndarray: Out-of-fold predictions.
        dict: Evaluation metrics.
    """
    # Perform CV
    kf = KFold(cv)
    scorers = {metric: get_scorer(metric)._score_func for metric in metrics}

    train_scores = {metric: [] for metric in metrics}
    valid_scores = {metric: [] for metric in metrics}
    oof_preds = np.zeros(len(y), dtype=int)
    models = []
    for fold, (tridx, validx) in enumerate(kf.split(X, y)):
        model_ = build_model(model_params)

        X_train, y_train = X[list(tridx)], y[list(tridx)]
        X_valid, y_valid = X[list(validx)], y[list(validx)]

        from sklearn.preprocessing import QuantileTransformer
        qtr = QuantileTransformer(n_quantiles=100)
        transformed_sq_footage = qtr.fit_transform(X_train['square_footage'].to_numpy().reshape(-1, 1))
        X_train = X_train.with_columns(square_footage=transformed_sq_footage)
        X_valid = X_valid.with_columns(square_footage=qtr.transform(X_valid['square_footage'].to_numpy().reshape(-1, 1)))

        # Train model
        # set sample weight 2 for properties having square footage more than 5000 else 1
        # sample_weight = np.where(X_train['square_footage'] > 5000, 5, 1)
        # model_.fit(X_train.to_numpy(), y_train.to_numpy().ravel(), sample_weight=sample_weight)
        model_.fit(X_train.to_numpy(), y_train.to_numpy().ravel())
        models.append(model_)

        # Predict on test dataset
        y_pred = model_.predict(X_valid.to_numpy())
        oof_preds[validx] = y_pred

        for metric in metrics:
            valid_score = scorers[metric](y_valid.to_numpy().ravel(), y_pred)
            valid_scores[metric].append(valid_score)

            train_score = scorers[metric](
                y_train.to_numpy().ravel(),
                model_.predict(X_train.to_numpy()),
            )
            train_scores[metric].append(train_score)

            # print(f"Fold: {fold+1}/{cv}, Train {metric}: {train_score:.3f}, Valid {metric}: {valid_score:.3f}")

    # Compute CV mean and standard deviation of train and valid scores
    cv_mean_train_scores = {
        metric: np.mean(train_scores[metric]) for metric in metrics
    }
    cv_std_train_scores = {
        metric: np.std(train_scores[metric]) for metric in metrics
    }
    cv_mean_valid_scores = {
        metric: np.mean(valid_scores[metric]) for metric in metrics
    }
    cv_std_valid_scores = {
        metric: np.std(valid_scores[metric]) for metric in metrics
    }

    # Compute OOF scores
    oof_score = {metric: scorers[metric](y, oof_preds) for metric in metrics}

    # Compute metrics on average of all models
    evaluation_metrics = {
        "CV Mean Train score": cv_mean_train_scores,
        "CV Std Train score": cv_std_train_scores,
        "CV Mean Valid score": cv_mean_valid_scores,
        "CV Std Valid score": cv_std_valid_scores,
        "OOF Score": oof_score,
    }

    return models, oof_preds, evaluation_metrics

In [ ]:
# Feature engineering
X = df.with_columns(
    (pl.col('bedrooms') + pl.col('bathrooms')).alias('B_plus_B'),
    (pl.col('bedrooms') * pl.col('bathrooms')).alias('B_prod_B'),
    (pl.col('bedrooms') / pl.col('bathrooms')).alias('B_div_B'),
    (pl.col('square_footage') / pl.col('bedrooms')).alias('sq_div_bed'),
    (pl.col('square_footage') / pl.col('bathrooms')).alias('sq_div_bath'),
    (pl.col('square_footage').median().over('zipcode')).alias('median_sq_ft_zipcode'),
    (pl.col('square_footage').mean().over('zipcode')).alias('mean_sq_ft_zipcode'),
    (pl.col('square_footage').std().over('zipcode')).alias('std_sq_ft_zipcode'),
    (pl.col('square_footage').min().over('zipcode')).alias('min_sq_ft_zipcode'),
    (pl.col('square_footage').max().over('zipcode')).alias('max_sq_ft_zipcode'),
    (pl.col('price').median().over('zipcode')).alias('median_price_zipcode'),
    (pl.col('price').mean().over('zipcode')).alias('mean_price_zipcode'),
    (pl.col('price').std().over('zipcode')).alias('std_price_zipcode'),
    (pl.col('price').min().over('zipcode')).alias('min_price_zipcode'),
    (pl.col('price').max().over('zipcode')).alias('max_price_zipcode'),
    ((pl.col('price') / pl.col('square_footage')).median().over('zipcode')).alias('median_price_per_sq_ft_zipcode'),
    ((pl.col('price') / pl.col('square_footage')).mean().over('zipcode')).alias('mean_price_per_sq_ft_zipcode'),
    ((pl.col('price') / pl.col('square_footage')).std().over('zipcode')).alias('std_price_per_sq_ft_zipcode'),
    ((pl.col('price') / pl.col('square_footage')).min().over('zipcode')).alias('min_price_per_sq_ft_zipcode'),
    ((pl.col('price') / pl.col('square_footage')).max().over('zipcode')).alias('max_price_per_sq_ft_zipcode'),
)

# One hot encode zip code
# X.drop_in_place('zipcode')
X = X.to_dummies('zipcode')

# Define feature columns
feature_cols = (
    ['bedrooms', 'bathrooms', 'square_footage']
      + [col for col in X.columns if col.startswith('zipcode')]
        + ['B_plus_B', 'B_prod_B', 'B_div_B', 'sq_div_bed', 'sq_div_bath',
            'median_sq_ft_zipcode', 'mean_sq_ft_zipcode', 'std_sq_ft_zipcode', 'min_sq_ft_zipcode', 'max_sq_ft_zipcode',
            'median_price_zipcode', 'mean_price_zipcode', 'std_price_zipcode', 'min_price_zipcode', 'max_price_zipcode',
            'median_price_per_sq_ft_zipcode', 'mean_price_per_sq_ft_zipcode', 'std_price_per_sq_ft_zipcode', 'min_price_per_sq_ft_zipcode', 'max_price_per_sq_ft_zipcode'])

target_col = 'price'

xgb_params = {
    'alpha': 13,
    'eta': 0.7,
    'max_depth': 8,
    'subsample': 0.5
}

# Train the model
models, oof_preds, evaluation_results = train(
    X=X[feature_cols],
    y=df[target_col],
    cv=5,
    model_params=xgb_params,
    metrics=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'],
)

In [ ]:
print(evaluation_results)

In [ ]:
fig = px.scatter(x=df[target_col], y=oof_preds, labels={'x': 'Ground Truth - Price', 'y': 'Predicted - Price'})
fig.add_trace(
    go.Scatter(x=df[target_col], y=df[target_col], name="linear", line_shape='linear')
)
fig.show()

In [ ]:
# Feature importance
feature_importances = sorted(list(zip(X[feature_cols].columns, np.mean([model.feature_importances_ for model in models], axis=0))), key=lambda x: x[1], reverse=True)
feature_importances_X = [x[0] for x in feature_importances if x[1] > 0][:20]
feature_importances_y = [x[1] for x in feature_importances if x[1] > 0][:20]

px.bar(x=feature_importances_X, y=feature_importances_y, labels={'x': 'Feature', 'y': 'Importance'}, title="Top 20 features by importance")

In [ ]:
import optuna

def objective(trial):
    # Hypertune decision tree
    model_params = {
        'alpha': trial.suggest_int('alpha', 0.0, 20.0),
        'eta': 0.7,
        'gamma': trial.suggest_int('gamma', 0, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'max_depth': trial.suggest_categorical('max_depth', [7,8,9,10,11,12,13,14,15,16,17,18]),
        'subsample': 0.5
    }

    _, _, metrics = train(
        X=X[feature_cols],
        y=df[target_col],
        model_params=model_params,
        cv=5,
        metrics=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'],
    )
    return metrics['OOF Score']['r2']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_params)

In [ ]:
from optuna.visualization import plot_slice
plot_slice(study)